In [1]:
# 0525
# BestModel = {6208: [6201, 'Binary'], 692: [6203, 'Africa'], 6203: [690, 'Binary'], 713: [6201, 'Africa'], 57: [6201, 'Binary'], 690: [6204, 'Africa'], 50: [58, 'Africa'], 51: [54, 'Binary'], 52: [6201, 'Binary'], 53: [53, 'Binary'], 54: [6201, 'Binary'], 55: [58, 'Africa'], 56: [52, 'Binary'], 6201: [6201, 'Africa'], 58: [53, 'Binary'], 59: [58, 'Binary'], 6204: [54, 'Africa'], 701: [6201, 'Africa']}

# 0601
BestModel = {6208: [690, 'Africa'], 692: [690, 'Africa'], 6203: [56, 'Binary'], 713: [690, 'Binary'], 57: [56, 'Binary'], 690: [52, 'Africa'], 50: [57, 'Binary'], 51: [6203, 'Binary'], 52: [690, 'Africa'], 53: [690, 'Africa'], 54: [690, 'Africa'], 55: [6201, 'Africa'], 56: [50, 'Binary'], 6201: [690, 'Binary'], 58: [6201, 'Binary'], 59: [56, 'Africa'], 6204: [56, 'Binary'], 701: [690, 'Africa']}

# 0608
BestModel = {6208: [690, 'Africa'], 692: [690, 'Africa'], 6203: [690, 'Africa'], 713: [690, 'Binary'], 57: [54, 'Binary'], 690: [6204, 'Africa'], 50: [690, 'Africa'], 51: [6203, 'Binary'], 52: [690, 'Africa'], 53: [690, 'Binary'], 54: [690, 'Africa'], 55: [57, 'Africa'], 56: [52, 'Binary'], 6201: [690, 'Binary'], 58: [54, 'Binary'], 59: [54, 'Binary'], 6204: [54, 'Binary'], 701: [52, 'Binary']}

_date_ = '20180608'
_model_date_ = '20180608'

In [2]:
import tensorflow as tf
from keras import backend as K
from keras.models import load_model

import math
import numpy as np
import pandas as pd
import sklearn.preprocessing as prep

import os
from datetime import date, timedelta, datetime

Using TensorFlow backend.


In [3]:
_scaler_ = prep.MinMaxScaler()

def scale_backward(d, y):
    if type(_scaler_) is prep.data.MinMaxScaler:
        return y * (max(d) - min(d)) + min(d)

def scale_toward(d, x):
    if type(_scaler_) is prep.data.MinMaxScaler:
        return (x - min(d)) / (max(d) - min(d))

In [4]:
def show_msg(now, last):
    diff = float( format(now, '.2f') ) - float( format(last, '.2f') )    
    if diff > 0:
        return "1"  # 漲
    elif diff < 0:
        return "-1" # 跌
    else:
        return "0"  # 平

def format_n_days(price):
    output = str(_code_) + '\t'
    for i in range(0, 5 if len(price)>=5 else len(price)):
        if i==0:
            output += '{}\t{:.2f}\t'.format(show_msg(price[i], last), price[i])
        else:
            output += '{}\t{:.2f}\t'.format(show_msg(price[i], price[i-1]), price[i])
    return output

def TBrain_score(predict_str, real_str):
    predict = predict_str.split('\t')[1:-1]
    real = real_str.split('\t')[1:-1]

    weights = [0.10, 0.15, 0.20, 0.25, 0.30]

    # (實際價格 – 絕對值(預測價格 – 實際價格)) /實際價格)*0.5
    p_score = [
        ( ( float(r) - abs(float(p) - float(r)) ) / float(r) ) * 0.5
        for (r, p) in zip(real[1::2], predict[1::2]) 
    ]

    # 預測正確得 0.5
    q_score = [
        0.5 if float(p) == float(r) else 0.0
        for (r, p) in zip(real[0::2], predict[0::2])
    ]

    return sum([ p*w + q*w for (p, q, w) in zip(p_score, q_score, weights) ])

In [5]:
_window_ = 30
_file_ = '18ETF.csv'

def tbrain_loss(y_true, y_pred):
    w = np.array([[0.10], [0.15], [0.20], [0.25], [0.30]])
    
    weights = np.concatenate((w,w,w,w,w,w))
    weights = tf.convert_to_tensor(weights, np.float32)
    
    squares = K.square(y_pred - y_true)
    squares = squares * weights
    
    return K.mean(squares, axis=-1)

In [6]:
df = pd.read_csv(os.path.join('/home/ddl/Desktop/Notebooks/TBrain/', _date_, _file_), thousands=',')

df['sort'] = df['代碼'].astype(str)
df = df.sort_values(by='sort', ascending=True).drop(columns=['sort'])

scoresA, scoresA_B, scores = [], [], []
    
for _code_ in df.代碼.unique():

    K.clear_session()

    model_Africa = load_model(
        os.path.join('/home/ddl/Desktop/Notebooks/Model/Africa-{}/{}_model_close.h5'.format(_model_date_, BestModel[_code_][0])),
        custom_objects={'tbrain_loss': tbrain_loss}
    )

    model_Binary = load_model(
        os.path.join('/home/ddl/Desktop/Notebooks/Model/Binary-{}/{}_model_close.h5'.format(_model_date_, BestModel[_code_][0])),
        custom_objects={'tbrain_loss': tbrain_loss}
    )

    data = df[df['代碼'] == _code_].sort_values(by='日期', ascending=True) \
                                    .drop(columns=['代碼', '日期', '中文簡稱'])

    last = np.array(data)[-1][3]
    date = int(df[df['代碼'] == _code_].sort_values(by='日期', ascending=False).head(1).日期)

    # predict 5 days
    dataArr = np.array(data)
    X = dataArr[-_window_:]
    X[:, 0] = scale_toward(list(dataArr[:, 0]), X[:, 0])
    X[:, 1] = scale_toward(list(dataArr[:, 1]), X[:, 1])
    X[:, 2] = scale_toward(list(dataArr[:, 2]), X[:, 2])
    X[:, 3] = scale_toward(list(dataArr[:, 3]), X[:, 3])
    X[:, 4] = scale_toward(list(dataArr[:, 4]), X[:, 4])
    X = np.expand_dims(X, axis=0)

    predictA = model_Africa.predict(X).reshape( X.shape[0], X.shape[1] )
    predictA = list(predictA[0][-5:])
    predictA = [ scale_backward(list(dataArr[:, 3]), i) for i in predictA ]

    predictB = model_Binary.predict(X)
    predictB = [[1, -1, 0][np.argmax(i)] for i in predictB[0][-5:].tolist()]

    predictA_B = format_n_days(predictA).split('\t')
    predictA_B[1::2] = [str(i) for i in predictB+['']]
    predictA_B = '\t'.join(predictA_B)
    
    if BestModel[_code_][1] == 'Africa':
        print(format_n_days(predictA) + '(Africa)')
    else:
        print(predictA_B + '(Africa + Binary)')

    # print real
    real_date = (datetime.strptime(_date_, '%Y%m%d')  + timedelta(days=7)).strftime('%Y%m%d')
    real_path = '/home/ddl/Desktop/Notebooks/TBrain/' + real_date + '/18ETF.csv'
    if os.path.exists(real_path):

        df2 = pd.read_csv(real_path, thousands=',')
        df2 = df2[df2['代碼'] == _code_]
        df2 = list( df2[df2['日期'] > date]['收盤價(元)'] )

        scoreA = TBrain_score(format_n_days(predictA), format_n_days(df2))
        scoreA_B = TBrain_score(predictA_B, format_n_days(df2))
            
        scoresA.append(scoreA)
        scoresA_B.append(scoreA_B)
        
        scores.append( scoreA if BestModel[_code_][1] == 'Africa' else scoreA_B )
        
        print(format_n_days(df2) + '(real)')
        print('{:.2f} {:.2f}'.format(scoreA, scoreA_B))


if scores != []:
    print('scoresA: {:.7f}'.format(sum(scoresA)))
    print('scoresA_B: {:.7f}'.format(sum(scoresA_B)))
    print('scores: {:.7f}'.format(sum(scores)))

50	-1	80.76	1	81.24	1	81.48	1	81.58	-1	81.32	(Africa)
51	-1	31.64	-1	31.74	-1	32.22	-1	32.54	-1	32.34	(Africa + Binary)
52	1	55.09	1	55.28	1	55.46	1	55.64	-1	55.46	(Africa)
53	1	36.75	1	36.93	1	37.09	1	37.16	1	37.11	(Africa + Binary)
54	1	26.63	1	26.76	1	26.94	1	27.22	1	27.29	(Africa)
55	-1	17.04	1	17.27	-1	17.26	1	17.47	-1	17.21	(Africa)
56	1	24.83	1	24.96	1	25.20	1	25.31	1	25.18	(Africa + Binary)
57	-1	48.69	-1	48.98	-1	49.53	-1	49.77	-1	48.99	(Africa + Binary)
58	-1	46.44	-1	46.36	-1	46.87	-1	47.50	-1	47.45	(Africa + Binary)
59	-1	40.50	-1	40.91	-1	41.12	-1	41.40	-1	41.24	(Africa + Binary)
6201	1	14.50	1	14.53	1	14.58	1	14.64	1	14.37	(Africa + Binary)
6203	-1	38.44	1	38.84	1	39.13	1	39.49	-1	39.43	(Africa)
6204	-1	52.53	-1	52.80	-1	53.90	-1	54.06	-1	54.03	(Africa + Binary)
6208	-1	47.82	1	48.23	1	48.52	1	48.86	-1	48.76	(Africa)
690	-1	20.24	-1	20.17	1	21.27	1	21.67	-1	21.47	(Africa)
692	-1	17.98	1	18.44	1	18.94	1	19.54	1	19.63	(Africa)
701	-1	16.80	1	17.69	1	18.86	1	19.69	1	18.75	(A